In [ ]:
#Set the path to the assignment folder in GDrive
from google.colab import drive

try:
  drive.mount('/content/gdrive')
except:
  print("Load the drive manually from the left panel first. Then run this again,")
  pathBase="/content/drive/MyDrive/"

Mounted at /content/gdrive


In [ ]:
!pip install transformers
!pip install torch
!pip install gensim

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 55.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 37.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 97.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 86.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 42.4 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import torch
import itertools
import copy
import numpy as np
import gensim.downloader as api
import time

from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score

from torch.utils.data import Dataset, DataLoader
from transformers import BertTokenizer
from sklearn.model_selection import train_test_split
from transformers import AdamW, DistilBertTokenizer, DistilBertForSequenceClassification
from transformers import RobertaTokenizer, RobertaForSequenceClassification
from torch.utils.data import DataLoader
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

#Data processing

In [ ]:
dataset = pd.read_csv("/content/gdrive/MyDrive/5000TravelQuestionsDataset.csv",encoding='latin-1')
# Drop the unnecessary column
dataset.drop(columns=["TTDSIG"], inplace=True)
column_names = ['question', 'coarse_grain_class']
dataset= pd.DataFrame(dataset)
dataset.columns = column_names
dataset.columns
dataset

,question,coarse_grain_class
0,What are the companies which organize shark fe...,TTD
1,Is it safe for female traveller to go alone to...,TGU
2,What are the best places around Cape Town for ...,TTD
3,What are the best places to stay for a family ...,ACM
4,What are the train services that travels from ...,TRS
...,...,...
4994,What is the best area to be based for sightsee...,TTD
4995,What are the good value traditional bars and r...,FOD
4996,What are the hotels near Alicante bus station?,ACM
4997,Where to stay in La Gomera to mountain biking?,TTD


In [ ]:
data = dataset[['question', 'coarse_grain_class']]
class_to_index = {"TTD": 0, "TGU": 1, "ACM": 2, "TRS": 3, "WTH": 4, "FOD": 5, "ENT": 6}

data['coarse_grain_class'] = data['coarse_grain_class'].str.strip()

data['coarse_grain_class'] = data['coarse_grain_class'].replace(class_to_index)

train_data, temp_data = train_test_split(data, train_size=4000, stratify=data['coarse_grain_class'], random_state=42)
test_data, val_data = train_test_split(temp_data, train_size=700, stratify=temp_data['coarse_grain_class'], random_state=42)



In [ ]:
class TextClassificationDataset(Dataset):
    def __init__(self, data, tokenizer, max_length):
        self.data = data
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.data)

    def __getitem__(self, index):
        text = self.data.iloc[index]['question']
        label = self.data.iloc[index]['coarse_grain_class']

        encoding = self.tokenizer.encode_plus(
    text,
    add_special_tokens=True,
    max_length=self.max_length,
    return_token_type_ids=False,
    padding='max_length',
    truncation=True,
    return_attention_mask=True,
    return_tensors='pt',
)


        return {
            'question_text': text,
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'labels': torch.tensor(label, dtype=torch.long)
        }


In [ ]:
def create_data_loader(data, tokenizer, max_len, batch_size):
    ds = TextClassificationDataset(
        data=data,
        tokenizer=tokenizer,
        max_length=max_len
    )

    return DataLoader(
        ds,
        batch_size=batch_size,
        num_workers=2
    )

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
max_len = 128
batch_size = 32


train_data_loader = create_data_loader(train_data, tokenizer, max_len, batch_size)
val_data_loader = create_data_loader(val_data, tokenizer, max_len, batch_size)
test_data_loader = create_data_loader(test_data, tokenizer, max_len, batch_size)


#LR (embedding layer)

In [ ]:
import torch.nn as nn

class LogisticRegressionEmbedding(nn.Module):
    def __init__(self, vocab_size, embedding_dim, output_dim):
        super(LogisticRegressionEmbedding, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.linear = nn.Linear(embedding_dim, output_dim)

    def forward(self, text):
        embedded = self.embedding(text).mean(dim=1)
        return self.linear(embedded)


In [ ]:
vocab_size = len(tokenizer.vocab)
embedding_dim = 100
output_dim = len(class_to_index)

model = LogisticRegressionEmbedding(vocab_size, embedding_dim, output_dim)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)


In [ ]:
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
import time

def train_epoch(model, data_loader, optimizer, criterion):
    model = model.train()
    losses = []
    all_preds = []
    all_labels = []

    for d in data_loader:
        inputs = d['input_ids'].to(device)
        labels = d['labels'].to(device)


        outputs = model(inputs)


        loss = criterion(outputs, labels)
        losses.append(loss.item())


        optimizer.zero_grad()
        loss.backward()
        optimizer.step()


        preds = torch.argmax(outputs, dim=1)
        all_preds.extend(preds.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())

    accuracy, (precision, recall, f1, _) = accuracy_score(all_labels, all_preds), precision_recall_fscore_support(all_labels, all_preds, average='weighted', zero_division=0)

    return np.mean(losses), accuracy, precision, recall, f1


In [ ]:
def evaluate_model(model, data_loader, criterion):
    model = model.eval()
    losses = []
    all_preds = []
    all_labels = []

    with torch.no_grad():
        for d in data_loader:
            inputs = d['input_ids'].to(device)
            labels = d['labels'].to(device)

            outputs = model(inputs)
            loss = criterion(outputs, labels)
            losses.append(loss.item())

            preds = torch.argmax(outputs, dim=1)
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    accuracy, (precision, recall, f1, _) = accuracy_score(all_labels, all_preds), precision_recall_fscore_support(all_labels, all_preds, average='weighted', zero_division=0)

    return np.mean(losses), accuracy, precision, recall, f1


In [ ]:


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)


best_model_state = None
best_metrics = {'accuracy': 0, 'precision': 0, 'recall': 0, 'f1': 0, 'time': float('inf')}

num_epochs = 10
start_time = time.time()

for epoch in range(num_epochs):
    train_loss, train_acc, train_prec, train_rec, train_f1 = train_epoch(model, train_data_loader, optimizer, criterion)
    val_loss, val_acc, val_prec, val_rec, val_f1 = evaluate_model(model, val_data_loader, criterion)

    if val_acc > best_metrics['accuracy']:
        best_metrics['accuracy'] = val_acc
        best_metrics['precision'] = val_prec
        best_metrics['recall'] = val_rec
        best_metrics['f1'] = val_f1
        best_model_state = copy.deepcopy(model.state_dict())

    print(f'Epoch {epoch + 1}/{num_epochs}')
    print(f'Train loss = {train_loss:.4f}, accuracy = {train_acc:.4f}, precision = {train_prec:.4f}, recall = {train_rec:.4f}, F1 = {train_f1:.4f}')
    print(f'Val loss = {val_loss:.4f}, accuracy = {val_acc:.4f}, precision = {val_prec:.4f}, recall = {val_rec:.4f}, F1 = {val_f1:.4f}')
    print('-' * 50)

end_time = time.time()
execution_time = end_time - start_time
best_metrics['time'] = execution_time

print(f'Training and evaluation time: {execution_time:.2f} seconds')
print('Best Results:')
print('Accuracy: {:.4f}, Precision: {:.4f}, Recall: {:.4f}, F1: {:.4f}, Time: {:.2f}s'.format(
    best_metrics['accuracy'],
    best_metrics['precision'],
    best_metrics['recall'],
    best_metrics['f1'],
    best_metrics['time']
))


Epoch 1/10
Train loss = 1.7892, accuracy = 0.2397, precision = 0.1694, recall = 0.2397, F1 = 0.1638
Val loss = 1.7856, accuracy = 0.2441, precision = 0.0596, recall = 0.2441, F1 = 0.0958
--------------------------------------------------
Epoch 2/10
Train loss = 1.7742, accuracy = 0.2542, precision = 0.2255, recall = 0.2542, F1 = 0.1769
Val loss = 1.7777, accuracy = 0.2441, precision = 0.0596, recall = 0.2441, F1 = 0.0958
--------------------------------------------------
Epoch 3/10
Train loss = 1.7648, accuracy = 0.2632, precision = 0.2480, recall = 0.2632, F1 = 0.1901
Val loss = 1.7676, accuracy = 0.2441, precision = 0.0596, recall = 0.2441, F1 = 0.0958
--------------------------------------------------
Epoch 4/10
Train loss = 1.7524, accuracy = 0.2792, precision = 0.2642, recall = 0.2792, F1 = 0.2131
Val loss = 1.7542, accuracy = 0.2441, precision = 0.0596, recall = 0.2441, F1 = 0.0958
--------------------------------------------------
Epoch 5/10
Train loss = 1.7361, accuracy = 0.308

#LR(pre-trained word embeddings)

In [ ]:
word_vectors = api.load("fasttext-wiki-news-subwords-300")

[==================================================] 100.0% 958.5/958.4MB downloaded


In [ ]:
vocab_size = len(tokenizer.vocab)
embed_dim = word_vectors.vector_size

embedding_matrix = np.zeros((vocab_size, embed_dim))

for word, idx in tokenizer.vocab.items():
    if word in word_vectors.key_to_index:
        embedding_matrix[idx] = word_vectors.get_vector(word)

In [ ]:
class LogisticRegressionPretrainedEmbeddings(nn.Module):
    def __init__(self, embed_matrix, num_classes):
        super(LogisticRegressionPretrainedEmbeddings, self).__init__()
        self.embedding = nn.Embedding.from_pretrained(torch.tensor(embed_matrix, dtype=torch.float32))
        self.linear = nn.Linear(embed_matrix.shape[1], num_classes)

    def forward(self, input_ids, attention_mask=None):
        embedded = self.embedding(input_ids).mean(dim=1)
        return self.linear(embedded)


In [ ]:
model = LogisticRegressionPretrainedEmbeddings(embedding_matrix, num_classes=len(class_to_index))
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)
criterion = nn.CrossEntropyLoss().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

In [ ]:
num_epochs = 10

for epoch in range(num_epochs):
    model.train()
    total_loss = 0
    for batch in train_data_loader:
        optimizer.zero_grad()

        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)

        outputs = model(input_ids, attention_mask)
        loss = criterion(outputs, labels)

        total_loss += loss.item()
        loss.backward()
        optimizer.step()

    avg_loss = total_loss / len(train_data_loader)
    print(f'Epoch {epoch+1}/{num_epochs}, Loss: {avg_loss:.4f}')


Epoch 1/10, Loss: 1.9032
Epoch 2/10, Loss: 1.8492
Epoch 3/10, Loss: 1.8145
Epoch 4/10, Loss: 1.7917
Epoch 5/10, Loss: 1.7763
Epoch 6/10, Loss: 1.7653
Epoch 7/10, Loss: 1.7571
Epoch 8/10, Loss: 1.7505
Epoch 9/10, Loss: 1.7450
Epoch 10/10, Loss: 1.7402


In [ ]:

def evaluate_model(model, data_loader):
    model.eval()
    all_preds = []
    all_labels = []

    start_time = time.time()

    with torch.no_grad():
        for batch in data_loader:
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['labels'].to(device)

            outputs = model(input_ids, attention_mask)
            _, preds = torch.max(outputs, dim=1)

            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    end_time = time.time()
    time_taken = end_time - start_time

    accuracy = accuracy_score(all_labels, all_preds)
    precision = precision_score(all_labels, all_preds, average='weighted', zero_division=0)
    recall = recall_score(all_labels, all_preds, average='weighted', zero_division=0)
    f1 = f1_score(all_labels, all_preds, average='weighted', zero_division=0)

    return accuracy, precision, recall, f1, time_taken


In [ ]:
accuracy, precision, recall, f1, time_taken = evaluate_model(model, test_data_loader)
print(f'Accuracy: {accuracy:.4f}, Precision: {precision:.4f}, Recall: {recall:.4f}, F1: {f1:.4f}, Time(s): {time_taken:.2f}')


Accuracy: 0.2529, Precision: 0.2562, Recall: 0.2529, F1: 0.1131, Time(s): 1.04


#DistilBERT

In [ ]:
distilbert_tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')
distilbert_model = DistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased', num_labels=7)

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['pre_classifier.bias', 'pre_classifier.weight', 'classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
def train_model(model, data_loader, optimizer, device):
    model = model.train()
    total_loss = 0
    for batch in data_loader:
        optimizer.zero_grad()
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)
        outputs = model(
            input_ids=input_ids,
            attention_mask=attention_mask,
            labels=labels
        )
        loss = outputs.loss
        total_loss += loss.item()
        loss.backward()
        optimizer.step()
    return total_loss / len(data_loader)

In [ ]:
def evaluate_model(model, data_loader, device):
    model = model.eval()
    predictions, true_labels = [], []
    with torch.no_grad():
        for batch in data_loader:
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['labels'].to(device)
            outputs = model(
                input_ids=input_ids,
                attention_mask=attention_mask,
                labels=labels
            )
            logits = outputs.logits
            _, preds = torch.max(logits, dim=1)
            predictions.extend(preds)
            true_labels.extend(labels)
    predictions = torch.stack(predictions).cpu()
    true_labels = torch.stack(true_labels).cpu()
    return accuracy_score(true_labels, predictions), precision_recall_fscore_support(true_labels, predictions, average='weighted')

In [ ]:
import torch
from transformers import AdamW
from transformers import DistilBertForSequenceClassification

best_lr = 2e-05
best_batch_size = 16
best_epochs = 4
best_max_len = 256


device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')


train_data_loader = create_data_loader(train_data, tokenizer, best_max_len, best_batch_size)
val_data_loader = create_data_loader(val_data, tokenizer, best_max_len, best_batch_size)
test_data_loader = create_data_loader(test_data, tokenizer, best_max_len, best_batch_size)

model = DistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased', num_labels=7).to(device)
optimizer = AdamW(model.parameters(), lr=best_lr)

for epoch in range(best_epochs):
    train_loss = train_model(model, train_data_loader, optimizer, device)
    val_acc, _ = evaluate_model(model, val_data_loader, device)
    print(f'Epoch {epoch + 1}/{best_epochs}, LR: {best_lr}, Batch Size: {best_batch_size}, Max Len: {best_max_len}')
    print(f'Train loss: {train_loss}, Validation accuracy: {val_acc}')


test_acc, (test_precision, test_recall, test_f1, _) = evaluate_model(model, test_data_loader, device)
print(f'Final Test Results - LR: {best_lr}, Batch Size: {best_batch_size}, Max Len: {best_max_len}')
print(f'Test accuracy: {test_acc}, Precision: {test_precision}, Recall: {test_recall}, F1: {test_f1}')


Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['pre_classifier.bias', 'pre_classifier.weight', 'classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch 1/4, LR: 2e-05, Batch Size: 16, Max Len: 256
Train loss: 0.9364142631888389, Validation accuracy: 0.8595317725752508
Epoch 2/4, LR: 2e-05, Batch Size: 16, Max Len: 256
Train loss: 0.3052932653427124, Validation accuracy: 0.8896321070234113
Epoch 3/4, LR: 2e-05, Batch Size: 16, Max Len: 256
Train loss: 0.15879578193277122, Validation accuracy: 0.8762541806020067
Epoch 4/4, LR: 2e-05, Batch Size: 16, Max Len: 256
Train loss: 0.08496189789474011, Validation accuracy: 0.8896321070234113
Final Test Results - LR: 2e-05, Batch Size: 16, Max Len: 256
Test accuracy: 0.8828571428571429, Precision: 0.8841928354096703, Recall: 0.8828571428571429, F1: 0.8827982494681659


##Model 1(global FT)

In [ ]:
from transformers import DistilBertForSequenceClassification, AdamW
import torch


device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

best_lr = 2e-05
best_batch_size = 16
best_epochs = 4
best_max_len = 256



train_data_loader = create_data_loader(train_data, tokenizer, best_max_len, best_batch_size)
val_data_loader = create_data_loader(val_data, tokenizer, best_max_len, best_batch_size)
test_data_loader = create_data_loader(test_data, tokenizer, best_max_len, best_batch_size)


model = DistilBertForSequenceClassification.from_pretrained(
    'distilbert-base-uncased',
    num_labels=7
).to(device)


optimizer = AdamW(model.parameters(), lr=best_lr)


for epoch in range(best_epochs):

    train_loss = train_model(model, train_data_loader, optimizer, device)


    val_acc, val_report = evaluate_model(model, val_data_loader, device)

    print(f'Epoch {epoch + 1}/{best_epochs} | Train loss: {train_loss:.4f} | Validation accuracy: {val_acc:.4f}')


test_acc, (test_precision, test_recall, test_f1, _) = evaluate_model(model, test_data_loader, device)
print(f'Test Results | Accuracy: {test_acc:.4f}, Precision: {test_precision:.4f}, Recall: {test_recall:.4f}, F1: {test_f1:.4f}')


Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['pre_classifier.bias', 'pre_classifier.weight', 'classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch 1/4 | Train loss: 0.9623 | Validation accuracy: 0.8528
Epoch 2/4 | Train loss: 0.3100 | Validation accuracy: 0.8829
Epoch 3/4 | Train loss: 0.1613 | Validation accuracy: 0.8829
Epoch 4/4 | Train loss: 0.0893 | Validation accuracy: 0.8696
Test Results | Accuracy: 0.8929, Precision: 0.8945, Recall: 0.8929, F1: 0.8921


##Model 1 (feature-based FT)

In [ ]:
from transformers import DistilBertModel
import torch.nn as nn
import torch.optim as optim
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
import torch

class DistilBertClassifier(nn.Module):
    def __init__(self, distilbert_model):
        super(DistilBertClassifier, self).__init__()
        self.distilbert = distilbert_model
        self.classifier = nn.Linear(distilbert_model.config.dim, 7)  # Assuming 7 classes

    def forward(self, input_ids, attention_mask):
        distilbert_output = self.distilbert(input_ids=input_ids, attention_mask=attention_mask)
        hidden_state = distilbert_output[0]
        pooled_output = hidden_state[:, 0]
        logits = self.classifier(pooled_output)
        return logits

def evaluate_model(model, data_loader, device):
    model.eval()
    all_predictions = []
    all_true_labels = []

    with torch.no_grad():
        for batch in data_loader:
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['labels'].to(device)

            outputs = model(input_ids=input_ids, attention_mask=attention_mask)
            _, preds = torch.max(outputs, dim=1)

            all_predictions.extend(preds.cpu().numpy())
            all_true_labels.extend(labels.cpu().numpy())

    accuracy = accuracy_score(all_true_labels, all_predictions)
    precision, recall, f1, _ = precision_recall_fscore_support(all_true_labels, all_predictions, average='weighted')
    return accuracy, precision, recall, f1


distilbert_model = DistilBertModel.from_pretrained('distilbert-base-uncased')


model = DistilBertClassifier(distilbert_model).to(device)


optimizer = optim.Adam(model.parameters(), lr=5e-5)


for epoch in range(3):
    model.train()
    for batch in train_data_loader:

        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)

        optimizer.zero_grad()
        logits = model(input_ids=input_ids, attention_mask=attention_mask)


        loss_fn = nn.CrossEntropyLoss()
        loss = loss_fn(logits.view(-1, 7), labels.view(-1))


        loss.backward()
        optimizer.step()


    accuracy, precision, recall, f1 = evaluate_model(model, val_data_loader, device)
    print(f'Epoch {epoch + 1}/3 - Validation accuracy: {accuracy}, Precision: {precision}, Recall: {recall}, F1: {f1}')

Epoch 1/3 - Validation accuracy: 0.8561872909698997, Precision: 0.8637288498607839, Recall: 0.8561872909698997, F1: 0.8561901803885336
Epoch 2/3 - Validation accuracy: 0.8896321070234113, Precision: 0.8889529054373567, Recall: 0.8896321070234113, F1: 0.888452751675529
Epoch 3/3 - Validation accuracy: 0.9063545150501672, Precision: 0.9088875381971406, Recall: 0.9063545150501672, F1: 0.9053554358130566


# RoBERTa

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
roberta_tokenizer = RobertaTokenizer.from_pretrained('roberta-base')
roberta_model = RobertaForSequenceClassification.from_pretrained('roberta-base', num_labels=7).to(device)
optimizer = AdamW(roberta_model.parameters(), lr=2e-5)

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.weight', 'classifier.dense.bias', 'classifier.out_proj.weight', 'classifier.out_proj.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [ ]:
def train_model(model, data_loader, optimizer, device):
    model = model.train()
    total_loss = 0
    for batch in data_loader:
        optimizer.zero_grad()
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)
        outputs = model(
            input_ids=input_ids,
            attention_mask=attention_mask,
            labels=labels
        )
        loss = outputs.loss
        total_loss += loss.item()
        loss.backward()
        optimizer.step()
    return total_loss / len(data_loader)

In [ ]:
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

def evaluate_model(model, data_loader, device):
    model = model.eval()

    predictions = []
    real_labels = []

    with torch.no_grad():
        for batch in data_loader:
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['labels'].to(device)

            outputs = model(
                input_ids=input_ids,
                attention_mask=attention_mask
            )

            logits = outputs.logits
            _, preds = torch.max(logits, dim=1)

            predictions.extend(preds.cpu().numpy())
            real_labels.extend(labels.cpu().numpy())

    accuracy = accuracy_score(real_labels, predictions)
    precision, recall, f1, _ = precision_recall_fscore_support(real_labels, predictions, average='weighted')

    return accuracy, (precision, recall, f1)


In [ ]:

final_model = RobertaForSequenceClassification.from_pretrained('roberta-base', num_labels=7).to(device)
final_optimizer = AdamW(final_model.parameters(), lr=1e-5, correct_bias=False)


final_train_data = pd.concat([train_data, val_data])
final_train_data_loader = create_data_loader(final_train_data, tokenizer, max_len, 64)


for epoch in range(num_epochs):
    print(f'Epoch {epoch + 1}/{num_epochs}')
    train_loss = train_model(final_model, final_train_data_loader, final_optimizer, device)
    print(f'Train loss: {train_loss}')


test_acc, (test_precision, test_recall, test_f1) = evaluate_model(final_model, test_data_loader, device)
print(f'Test accuracy: {test_acc}, Precision: {test_precision}, Recall: {test_recall}, F1: {test_f1}')


Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.weight', 'classifier.dense.bias', 'classifier.out_proj.weight', 'classifier.out_proj.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch 1/10
Train loss: 1.6660118979566239
Epoch 2/10
Train loss: 1.3781105797080433
Epoch 3/10
Train loss: 1.1275722042602652
Epoch 4/10
Train loss: 0.9441857079372686
Epoch 5/10
Train loss: 0.7952002078733024
Epoch 6/10
Train loss: 0.656061605495565
Epoch 7/10
Train loss: 0.5251050961806494
Epoch 8/10
Train loss: 0.42463783749981837
Epoch 9/10
Train loss: 0.3766754827297786
Epoch 10/10
Train loss: 0.3380465805530548
Test accuracy: 0.7385714285714285, Precision: 0.750749962004738, Recall: 0.7385714285714285, F1: 0.7369962760921759


##Model 2(global FT)

In [ ]:



device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


tokenizer = RobertaTokenizer.from_pretrained('roberta-base')
model = RobertaForSequenceClassification.from_pretrained('roberta-base', num_labels=7).to(device)


for param in model.parameters():
    param.requires_grad = True


optimizer = AdamW(model.parameters(), lr=1e-5, correct_bias=False)


train_data_loader = create_data_loader(train_data, tokenizer, max_len, batch_size=64)
val_data_loader = create_data_loader(val_data, tokenizer, max_len, batch_size=64)
test_data_loader = create_data_loader(test_data, tokenizer, max_len, batch_size=64)


num_epochs = 3


for epoch in range(num_epochs):
    print(f'Epoch {epoch + 1}/{num_epochs}')


    train_loss = train_model(model, train_data_loader, optimizer, device)
    print(f'Train loss: {train_loss}')


    val_acc, (val_precision, val_recall, val_f1) = evaluate_model(model, val_data_loader, device)
    print(f'Validation Accuracy: {val_acc}, Precision: {val_precision}, Recall: {val_recall}, F1: {val_f1}')


test_acc, (test_precision, test_recall, test_f1) = evaluate_model(model, test_data_loader, device)
print(f'Test Accuracy: {test_acc}, Precision: {test_precision}, Recall: {test_recall}, F1: {test_f1}')


Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.weight', 'classifier.dense.bias', 'classifier.out_proj.weight', 'classifier.out_proj.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch 1/3


/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Train loss: 0.8580877270017352
Validation Accuracy: 0.8795986622073578, Precision: 0.8870033507981079, Recall: 0.8795986622073578, F1: 0.8803624746347699
Epoch 2/3
Train loss: 0.3209896191718086
Validation Accuracy: 0.8695652173913043, Precision: 0.8749869374149462, Recall: 0.8695652173913043, F1: 0.8678505150391596
Epoch 3/3
Train loss: 0.20345761248516658
Validation Accuracy: 0.8896321070234113, Precision: 0.8937145539221341, Recall: 0.8896321070234113, F1: 0.8892044962805075
Test Accuracy: 0.8971428571428571, Precision: 0.8970591051851988, Recall: 0.8971428571428571, F1: 0.896631595643447


#Model 2 (feature-based FT)

In [ ]:
from transformers import RobertaForSequenceClassification, AdamW


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


final_model = RobertaForSequenceClassification.from_pretrained('roberta-base', num_labels=7).to(device)


for param in final_model.parameters():
    param.requires_grad = False


for name, param in final_model.named_parameters():
    if 'classifier' in name or 'pooler' in name:
        param.requires_grad = True


final_optimizer = AdamW(filter(lambda p: p.requires_grad, final_model.parameters()), lr=1e-5)

final_train_data_loader = create_data_loader(final_train_data, tokenizer, max_len, 64)


num_epochs = 3
for epoch in range(num_epochs):
    print(f'Epoch {epoch + 1}/{num_epochs}')
    train_loss = train_model(final_model, final_train_data_loader, final_optimizer, device)
    print(f'Train loss: {train_loss}')


test_acc, (test_precision, test_recall, test_f1) = evaluate_model(final_model, test_data_loader, device)
print(f'Test accuracy: {test_acc}, Precision: {test_precision}, Recall: {test_recall}, F1: {test_f1}')


Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.weight', 'classifier.dense.bias', 'classifier.out_proj.weight', 'classifier.out_proj.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch 1/3


/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Train loss: 1.937366990482106
Epoch 2/3
Train loss: 1.8661636990659378
Epoch 3/3
Train loss: 1.8304068621467142
Test accuracy: 0.24428571428571427, Precision: 0.1105393609445708, Recall: 0.24428571428571427, F1: 0.10119718047435342


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
